# Modeling Patient-Initiated Timing to Minimize Waits at Vancouver Island Hospital's Emergency Departments Using a Queueing-Network Approach
Taylor Dew-Jones and Jay Robertson<br>
<i>CSC 446: Operations in Simulations at the University of Victoria</i>

**Note to ourselves, the arrivals and mean service times will need to be updated/edited as we get going. They SHOULD reflect approx 12 hours mean wait in the system, discounting patients who have a wait before discharge AFTER seeing the doctor at DIAGNOSIS stage. 

STATS TO TRACK: (basically what we've been using for the assignments): avg_num_patients for each queue and for sys, avg_delay_patients for each queue and for sys, most importantly we need to snapshot these at the different time intervals over multi-24 hours periods. area_num_in_queues each queue, area_staff_status (would this one per individual staff or just one for each queue for all staff at that queue for the interval)
  
24 hours snapshots, broken down by intervals, delays tracked hourly via list????

In [47]:
import numpy as np
import bisect
np.random.seed(3)

In [48]:
class Simulation:
    def __init__(self, num_days):
        
        #Initialize timing variables
        self.day_length = 24 * 60                       #minutes in a day
        self.sim_length = num_days * self.day_length    #sim end time [min]
        self.sim_time = 0                               #length of simulation so far [min]
        self.sim_time_day = 0                           #time of day for hospital [min]
        self.current_day = 1                            #start at day 1
        self.last_recorded_hour = -1                    #for hourly stats tracking

        #Initialize queues
        self.triage_Q = TriageQueue()
        self.xray_Q = TestingQueue(6, [1, 2, 1, 1], 1, [0, -1, -1]) #(mean_service, # of nurses during shifts, queue label)
        self.ecg_Q = TestingQueue(10, [1, 1, 1, 1], 2, [0, -1, -1])
        self.bloodwork_Q = TestingQueue(16, [3, 3, 3, 3], 3, [0, 0, 0])
        self.diagnosis_Q = DiagnosisQueue()

        #Initialize event variables
        self.next_event_type = 0
        self.next_index = 0

        self.waiting_queue = [] #discharge times of waiting patients, tracked (discharge_time, arrival_time)

        #system stats
        self.daily_total_time_in_sys = [0.0] * num_days
        self.area_num_in_system_daily = [0.0] * num_days
        self.daily_num_completed = [0] * num_days
        self.area_num_in_system = 0.0
        self.num_in_sys = 0         #current count
        self.time_last_event = 0.0  #time of last event

        #hourly stats
        self.area_daily_hourly_L = [ [0.0]*24 for _ in range(num_days) ] #L = avg num patients in sys: waiting in q + being served
        self.hourly_completed_times = [ [0.0]*24 for _ in range(num_days) ]  # sum of completed times per hour
        self.hourly_completed_counts = [ [0]*24 for _ in range(num_days) ]   # number of completions per hour

    def main(self):
        #Run the simulation for given number of days.
        while self.sim_time < self.sim_length:
            day_index = self.discharge_waitQ()   #discharge patients that are waiting
            self.timing()            #find earliest scheduled event

            '''DEBUG STATEMENT'''
            #print(f'sim_time={self.sim_time:.3f}, next_type={self.next_event_type}, queue: {self.triage_Q.time_next_event}, num={len(self.triage_Q.queue)}')
    
            #next event is from TRIAGE
            if self.next_event_type == 0:
                if self.next_index < 3:
                    is_assisted = True if self.next_index == 1 else False
                    self.num_in_sys += 1
                    self.triage_Q.arrive(self.sim_time, is_assisted)
                else:
                    arrival_time, priority = self.triage_Q.depart(self.next_index, self.sim_time)
    
                    #choose random testing queue
                    ran_testq = int(np.random.choice([1, 2, 3], p=[0.05, 0.45, 0.50]))
    
                    #if level 1 priority, reroll random test queue until not xray queue
                    while priority == 1 and ran_testq == 1: 
                        ran_testq = int(np.random.choice([1, 2, 3], p=[0.05, 0.45, 0.50]))
    
                    #schedule immediate arrival in the random testing queue
                    if ran_testq == 1:    self.xray_Q.arrive(self.sim_time, arrival_time, priority)
                    elif ran_testq == 2:  self.ecg_Q.arrive(self.sim_time, arrival_time, priority)
                    else:                 self.bloodwork_Q.arrive(self.sim_time, arrival_time, priority)

            #next event is from TESTING
            elif self.next_event_type in [1, 2, 3]: 
                if self.next_event_type == 1:     queue = self.xray_Q
                elif self.next_event_type == 2:   queue = self.ecg_Q
                else:                             queue = self.bloodwork_Q

                if self.next_index != 1: 
                    arrival_time, priority = queue.depart(self.next_index, self.sim_time)

                    #schedule immediate arrival in diagnosis queue
                    self.diagnosis_Q.arrive(self.sim_time, arrival_time, priority)
    
            #next event is from DIAGNOSIS
            else:
                if self.next_index != 1:
                    arrival_time, priority = self.diagnosis_Q.depart(self.next_index, self.sim_time)
                    hour = int((arrival_time % self.day_length) // 60)
                
                    is_discharged = int(np.random.choice([0, 1], p=[0.2, 0.8]))
                    if is_discharged:
                        t_sys = self.sim_time - arrival_time
                        self.daily_total_time_in_sys[day_index] += t_sys
                        self.area_num_in_system_daily[day_index] += t_sys**2
                        self.daily_num_completed[day_index] += 1
                        self.num_in_sys -= 1                       
                        self.hourly_completed_times[day_index][hour] += t_sys
                        self.hourly_completed_counts[day_index][hour] += 1
                    else:
                        waiting_len = np.random.uniform(3*60, 12*60)
                        discharge_time = self.sim_time + waiting_len
                        bisect.insort(self.waiting_queue, (discharge_time, arrival_time)) #to maintain an ordered list

            self.update_time_stats()

            #check if the day has ended
            while self.sim_time >= self.current_day * self.day_length:
                self.report()
                self.reset_day()

    def discharge_waitQ(self):
        '''check for any patients in waiting queue who are ready to be discharged'''
        day_index = self.current_day - 1
        
        while self.waiting_queue and self.waiting_queue[0][0] <= self.sim_time:
                discharge_time, arrival_time = self.waiting_queue.pop(0)
                t_sys = discharge_time - arrival_time

                hour = int((arrival_time % self.day_length) // 60)
                
                #update daily totals
                self.daily_total_time_in_sys[day_index] += t_sys
                self.area_num_in_system_daily[day_index] += t_sys**2
                self.daily_num_completed[day_index] += 1
                self.num_in_sys -= 1
            
                #update hourly totals
                self.hourly_completed_times[day_index][hour] += t_sys
                self.hourly_completed_counts[day_index][hour] += 1

        return day_index

    def timing(self):
        '''Determine earliest scheduled event in all queues and update tracking variables'''
        
        queue_with_min_event = None         #hospital queue with earliest event
        min_time_next_event = float('inf')  #earliest time_next_event in queue_with_min_event
        min_index = float('inf')

        queue_list = [self.triage_Q, self.xray_Q, self.ecg_Q, self.bloodwork_Q, self.diagnosis_Q]
        for queue in queue_list:
            queue_min_time = min(queue.time_next_event[1:]) #get earliest time in queue

            if queue_min_time < min_time_next_event: 
                min_time_next_event = queue_min_time
                queue_with_min_event = queue
                for i in range(len(queue.time_next_event)):
                    if queue.time_next_event[i] == queue_min_time:
                        min_index = i
                        break

        self.next_event_type = queue_with_min_event.queue_label
        self.next_index = min_index
        self.sim_time = min_time_next_event
        self.sim_time_day = self.sim_time % self.day_length     #computes the time within the current day

    def update_time_stats(self):
        """Update time-weighted averages for all queues and overall systems. Tracking hourly stats for later analysis. 
        Daily stats tracking is handled by printing the prev day's results before resetting stats counters for next day."""

        # Time since last event
        elapsed = self.sim_time - self.time_last_event
        if elapsed < 0: elapsed = 0  # safety check
        self.time_last_event = self.sim_time

        day_index = self.current_day - 1

        #update system averages
        self.area_num_in_system += self.num_in_sys * elapsed                      #overall sim
        self.area_num_in_system_daily[day_index] += self.num_in_sys * elapsed     #daily totals
        
        #the hourly snapshots
        current_hour = min(int(self.sim_time_day // 60), 23)
        self.area_daily_hourly_L[day_index][current_hour] += self.num_in_sys * elapsed
        
        #shift change
        self.shift_change_master(False)

    def shift_change_master(self, is_new_day):
        """Enforce shift changes based on the real shift list of each queue."""
    
        if self.sim_time == 0: return  # skip first moment of simulation
    
        queues = [self.triage_Q, self.xray_Q, self.diagnosis_Q]
        
        #if new day, reset shift index and apply first shift
        if is_new_day:
            for Q in queues:
                Q.cur_shift_index = 0
                Q.shift_change(self.sim_time)
    
            #reset triage arrival rate index
            self.triage_Q.cur_time_index = 0
            return

        #same-day staff shift change
        for Q in queues:
            if (Q.cur_shift_index + 1) < len(Q.shifts):
                if self.sim_time_day >= Q.shifts[Q.cur_shift_index + 1]:
                    Q.shift_change(self.sim_time)

                    '''DEBUGGING'''
                    #print(f'Shift change in {Q.queue_label}, sim_time={self.sim_time_day}, i={Q.cur_shift_index}, shifts={Q.shifts[Q.cur_shift_index]}')

        if self.triage_Q.cur_time_index + 1 < len(self.triage_Q.interarrival_shifts):
            next_shift = self.triage_Q.interarrival_shifts[self.triage_Q.cur_time_index + 1]

            if self.sim_time_day >= next_shift:
                self.triage_Q.cur_time_index += 1
                

    def reset_day(self):
        '''If simulation reaches midnight, reset 24-hour clock dependent values: We reset all daily stats and averages while preserving patient queues'''

        # Reset simulation tracking
        self.time_last_event = self.sim_time
        self.last_recorded_hour = -1

        #reset queue variables
        queues = [self.triage_Q, self.xray_Q, self.ecg_Q, self.bloodwork_Q, self.diagnosis_Q]
        for Q in queues:
            Q.avg_num_in_system = 0.0
        
        #increment the daily counters
        self.current_day += 1       #increment to the next day
        self.sim_time_day = 0.0     #reset the daily clock
        
        #Reset time-weighted average   
        self.avg_num_in_system_waiting = 0.0 
        self.shift_change_master(True)

    def report(self):
        """Print daily statistics for all queues and overall system. Hourly stats are tracking via list, for printing as DAILY."""
        day_index = self.current_day - 1

        print(f"\n--- Report for Day {self.current_day} ---")
        print(f"Total patients completed: {self.daily_num_completed[day_index]}")

        if self.daily_num_completed[day_index] > 0:
            avg_time_in_sys = self.daily_total_time_in_sys[day_index] / self.daily_num_completed[day_index]
            variance = (self.area_num_in_system_daily[day_index] / self.daily_num_completed[day_index]) - avg_time_in_sys**2
            sd_time_in_sys = max(variance**0.5, 0.0)
            #sd_time_in_sys = variance**0.5 if variance > 0 else 0.0
        else:
            avg_time_in_sys = 0.0
            sd_time_in_sys = 0.0

        print(f"Average time in system (minutes): {avg_time_in_sys:.2f}")
        print(f"Standard deviation of time in system (minutes): {sd_time_in_sys:.2f}")

        # Hourly snapshots
        print("\nHourly snapshots:")
        for hour in range(24):
            count = self.hourly_completed_counts[day_index][hour]
            avg_wait = self.hourly_completed_times[day_index][hour] / count if count > 0 else 0.0
            print(f"Hour {hour}: num in sys: {self.area_daily_hourly_L[day_index][hour]/60:.2f} ; avg wait time of patients arriving at hour = {avg_wait:.2f} min")


## Stage 1 of the queue system, TRIAGE  
  
We will use Exponential distribution to simulate patients arrival at the hospital. There will be two separate exp distros generated: one to sim unassisted arrivals (patient walks in) and one for assisted arrivals (patients brought in via ambulance). Patient will be randomly assigned a priority level as they are generated. There is just one FIFO queue that they all join, EXCEPT FOR PATIENTS WITH LEVEL 1 PROPRITY, WHO WILL JOIN AT THE FRONT OF THE QUEUE.  
Level 1 is the highest priority, and 4 is the lowest. They should be at this mean distribution: 1: 3% ; 2: 42% ; 3: 45% ; 4: 10%.  
| | 00:00 to 03:00 | 03:01 to 05:00 | 05:01 to 11:00 | 11:01 to 22:00 | 22:01 to 23:59 |
|:---------------------------------:|:----------------:|:----------------:|:----------------:|:----------------:|:----------------:|
|$\lambda_1$ for unassisted patients| 12 / hr | 1 / hr | 14 / hr | 16 / hr | 12 / hr |
|$\lambda_2$ for assisted patients  | 8 / hr | 2 / hr | 6 / hr | 4 / hr | 8 / hr |
  
The triage queue staffing levels (mean service time: 12 patients / hr per nurse).:  
| | 00:00 to 03:00 | 03:01 to 07:00 | 07:01 to 19:00 | 19:01 to 23:59 | 
|:-------------------:|:----------------:|:----------------:|:----------------:|:----------------:|
|num nurses at TRIAGE | 3 | 1 | 2 | 3 |

If an assisted patient and an unassisted patient arrive at the same time, assisted patient will be served first.

In [49]:
class TriageQueue:
    def __init__(self):
        self.queue_label = 0 #unique identification number

        self.queue = [] #combined time_arrival and priorities

        #Initialize interarrival info
        self.cur_time_index = 0
        #self.interarrival_shifts = [0, (3*60)+1, (5*60)+1, (11*60)+1, (22*60)+1] #[min]
        #self.unass_interarrivals = np.array([1/12, 1, 1/14, 1/16, 1/12]) * 60            #[min/customer]
        #self.ass_interarrivals = np.array([1/8, 1/2, 1/6, 1/4, 1/8]) * 60                #[min/customer]
        self.interarrival_shifts = [0, (3*60)+1, (7*60)+1, (19*60)+1] #[min]
        self.unass_interarrivals = np.array([1/12, 1, 1/14, 1/16]) * 60 * 1.25           #[min/customer]
        self.ass_interarrivals = np.array([1/8, 1/2, 1/6, 1/4]) * 60 * 1.25

        #Pre-schedule first arrivals to avoid timing conflict
        first_ass = self.expon(self.ass_interarrivals[0])
        first_unass = self.expon(self.unass_interarrivals[0])

        #[empty, assisted arrive, unassisted arrive, depart 1, depart 2, depart 3]
        self.time_next_event = [0, first_ass, first_unass, float('inf'), float('inf'), float('inf')]

        #Initialize nurse values
        self.mean_service = 60/12             #[patients/min]
        self.nurse_status = [0, 0, 0]         #0: idle, 1: busy, -1: not available
        self.num_nurses = [1, 1, 2, 3]        #number of nurses during each shift
        self.shifts = [0, 181, 421, 1141]     #time for shift changes [min]
        self.cur_shift_index = 0              #index of last shift change
        self.current_patients = [-1, -1, -1]  #patients being serviced by nurses, entries: (arrival_time, priority)

    def get_priority(self):
         '''priorities randomly assigned to patients based on probability'''
         return int(np.random.choice([1, 2, 3, 4], p=[0.03, 0.42, 0.45, 0.10]))

    def shift_change(self, sim_time):
        self.cur_shift_index = (self.cur_shift_index+1)%4
        nurses_available = self.num_nurses[self.cur_shift_index]

        #change number of nurses available
        for i in range(3):
            if i < nurses_available:
                if self.nurse_status[i] == -1:
                    self.nurse_status[i] = 0  #make newly available nurse idle
            else:
                self.nurse_status[i] = -1  #nurse not available

        # Assign patients from queue to newly available nurses
        for i, status in enumerate(self.nurse_status):
            if status == 0 and len(self.queue) > 0:
                arrival_time, priority = self.queue.pop(0)
                self.time_next_event[3 + i] = sim_time + self.expon(self.mean_service)
                self.nurse_status[i] = 1
                self.current_patients[i] = (arrival_time, priority)

    def arrive(self, sim_time, is_assisted):
        priority = self.get_priority() 
        
        #schedule next arrival
        if is_assisted:
            self.time_next_event[1] = sim_time + self.expon(self.ass_interarrivals[self.cur_time_index])
        else:
            self.time_next_event[2] = sim_time + self.expon(self.unass_interarrivals[self.cur_time_index])

        #if a nurse is idle
        if 0 in self.nurse_status:
            nurse_index = self.nurse_status.index(0) #find index of first idle server
            self.nurse_status[nurse_index] = 1
            self.time_next_event[3+nurse_index] = sim_time + self.expon(self.mean_service)
            self.current_patients[nurse_index] = (sim_time, priority)
        
        #if all nurses busy
        else:
            if priority == 1:
                self.queue.insert(0, (sim_time, priority))
            else:
                self.queue.append((sim_time, priority))

    def depart(self, event_index, sim_time):
        arrival_time = self.current_patients[event_index-3][0]
        priority = self.current_patients[event_index-3][1]
        
        #if queue is empty
        if len(self.queue) == 0:
            if self.nurse_status[event_index-3] != -1:
                self.nurse_status[event_index-3] = 0
                self.current_patients[event_index-3] = -1
                
            self.time_next_event[event_index] = float('inf')
            
            return arrival_time, priority

        #schedule departure if queue has patients
        if self.nurse_status[event_index-3] != -1: 
            new_arrival_time, new_priority = self.queue.pop(0)
            self.current_patients[event_index-3] = (new_arrival_time, new_priority)
            self.time_next_event[event_index] = sim_time + self.expon(self.mean_service)
        else:
            #if shift change removed nurse, don't schedule departure
            self.time_next_event[event_index] = float('inf')
            
        return arrival_time, priority
        
    def expon(self, mean):
        """Function to generate exponential random variates."""

        return -mean * np.log(np.random.uniform(0, 1))
                       

## Stage 2 of the queue system, TESTS  
As patients leave the triage queue they will be randomly assigned to a test queue (all priority queues), based on probability: Xray: 5%; ECG: 45%; Bloodwork: 50%. ALL ECG PATIENTS ALSO GET BLOODWORK, AFTER.  
Note that patients with level 1 priority will NEVER be assigned to the Xray queue.  
  
Mean service times of the 3 different tests:  
| | avg $\mu$ per staff| 
|:---------------------------------:|:----------------:|
|Xray| 6 / hr |
|ECG  | 10 / hr |
|Bloodwork  | 16 / hr |
  
The tests staffing levels (mean service time as above, per staff):  
| | 00:00 to 03:00 | 03:01 to 07:00 | 07:01 to 19:00 | 19:01 to 23:59 | 
|:-------------------:|:----------------:|:----------------:|:----------------:|:----------------:|
| Xray | 1 | 2 | 1 | 1 |
| ECG | 1 | 1 | 1 | 1 |
|Bloodwork | 3 | 3 | 3 | 3 |
  

In [50]:
class TestingQueue:
    def __init__(self, mean_service, staffing, label, nurse_status):
        self.queue_label = label #unique identification number

        #Initialize time values
        self.time_next_event = [0, float('inf'), float('inf'), float('inf'), float('inf')] #[<not used>, arrival, departures 1-3]
        self.queue = [] #replace time_arrival with the queue, stores (arrival_time, priority)

        #Initialize nurse values
        self.cur_shift_index = 0
        self.mean_service = 60/mean_service   #[patients/min]
        self.num_nurses = staffing            #num nurses during each shift
        self.shifts = [0, 181, 421, 1141]      #time for shift changes [min]
        self.nurse_status = nurse_status     #0: idle, 1: busy, -1: not available
        self.current_patients = [-1, -1, -1]  #patients being serviced by nurses, entries: (arrival_time, priority)

    def shift_change(self, sim_time):
        self.cur_shift_index = (self.cur_shift_index+1)%4
        nurses_available = self.num_nurses[self.cur_shift_index]

        #change number of nurses available
        for i in range(3):
            if i < nurses_available:
                if self.nurse_status[i] == -1:
                    self.nurse_status[i] = 0  #make newly available nurse idle
            else:
                self.nurse_status[i] = -1  #nurse not available

        # Assign patients from queue to newly available nurses
        for i, status in enumerate(self.nurse_status):
            if status == 0 and len(self.queue) > 0:
                arrival_time, priority = self.queue.pop(0)
                self.time_next_event[2 + i] = sim_time + self.expon(self.mean_service)
                self.nurse_status[i] = 1
                self.current_patients[i] = (arrival_time, priority)

    def arrive(self, sim_time, arrival_time, priority):
        patient_data = (arrival_time, priority)
        
        #if a nurse is idle
        if 0 in self.nurse_status:
            nurse_index = self.nurse_status.index(0) #find index of first idle server
            self.nurse_status[nurse_index] = 1
            self.time_next_event[2+nurse_index] = sim_time + self.expon(self.mean_service)
            self.current_patients[nurse_index] = patient_data
        
        #if all nurses busy
        else:
            if priority == 1:
                self.queue.insert(0, patient_data)
            else:
                self.queue.append(patient_data)

    def depart(self, event_index, sim_time):
        nurse_index = event_index-2
        
        arrival_time = self.current_patients[nurse_index][0]
        priority = self.current_patients[nurse_index][1]
        
        #if queue is empty
        if len(self.queue) == 0:
            if self.nurse_status[nurse_index] != -1:
                self.nurse_status[nurse_index] = 0
                self.current_patients[nurse_index] = -1
                
            self.time_next_event[event_index] = float('inf')
            
            return arrival_time, priority

        #schedule departure if queue has patients
        if self.nurse_status[nurse_index] != -1: 
            new_arrival_time, new_priority = self.queue.pop(0)
            self.current_patients[nurse_index] = (new_arrival_time, new_priority)
            self.time_next_event[event_index] = sim_time + self.expon(self.mean_service)
        else:
            #if shift change removed nurse, don't schedule departure
            self.time_next_event[event_index] = float('inf')
            
        return arrival_time, priority

    def expon(self, mean):
        """Function to generate exponential random variates."""

        return -mean * np.log(np.random.uniform(0, 1))
                       

## Stage 3 of the queue system, DIAGNOSIS

Doctor staffing levels (mean service time of 5 / hr, per staff):  
| | 00:00 to 05:00 | 05:01 to 06:00 | 06:01 to 07:00 | 07:01 to 16:00 | 16:01 to 17:00 | 17:01 to 23:59 |
|:-------------------:|:----------------:|:----------------:|:----------------:|:----------------:|:----------------:|:----------------:|
| num doctors at DIAGNOSIS | 1 | 2 | 3 | 4 | 3 | 2 |
  
After diagnosis, patients are either discharged (80%) or (20%) are told to wait(x) for a random amount of time, uniformly distributed between x = 3 to 12 hours. 

In [51]:
class DiagnosisQueue:
    def __init__(self):
        self.queue_label = 4 #unique identification number
        
        #[<not used>, arrival, departure 1, departure 2, departure 3, departure 4]
        self.time_next_event = [0, float('inf'), float('inf'), float('inf'), float('inf'), float('inf')] 
        
        self.queue = [] #(arrival_time, priority)

        #Initialize doctor values
        self.mean_service = 60/5                                                 #[patients/min]
        self.doctor_status = [0, -1, -1, -1]                                     #0: idle, 1: busy, -1: not available
        self.num_doctors = [1, 2, 3, 4, 3, 2]                                    #num doctors during each shift
        self.shifts = [0, (5*60)+1, (6*60)+1, (7*60)+1, (16*60)+1, (17*60)+1]    #time for shift changes [min]
        self.cur_shift_index = 0                                                  #index of last shift change
        self.current_patients = [-1, -1, -1, -1]  #patients being serviced by doctors, entries: (arrival_time, priority)

    def shift_change(self, sim_time):
        self.cur_shift_index = (self.cur_shift_index+1)%6
        doctors_available = self.num_doctors[self.cur_shift_index]

        #change number of nurses available
        for i in range(4):
            if i < doctors_available:
                if self.doctor_status[i] == -1:
                    self.doctor_status[i] = 0  #make newly available doctor idle
            else:
                self.doctor_status[i] = -1  #doctor not available

        # Assign patients from queue to newly available doctors
        for i, status in enumerate(self.doctor_status):
            if status == 0 and len(self.queue) > 0:
                arrival_time, priority = self.queue.pop(0)
                self.time_next_event[2 + i] = sim_time + self.expon(self.mean_service)
                self.doctor_status[i] = 1
                self.current_patients[i] = (arrival_time, priority)
            
    def arrive(self, sim_time, arrival_time, priority):
        patient_data = (arrival_time, priority)
        
        #if a doctor is idle
        if 0 in self.doctor_status:
            doctor_index = self.doctor_status.index(0) #find index of first idle server
            self.doctor_status[doctor_index] = 1
            self.time_next_event[2+doctor_index] = sim_time + self.expon(self.mean_service)
            self.current_patients[doctor_index] = patient_data
        
        #if all nurses busy
        else:
            if priority == 1:
                self.queue.insert(0, patient_data)
            else:
                self.queue.append(patient_data)

    def depart(self, event_index, sim_time):
        doctor_index = event_index-2
        
        arrival_time = self.current_patients[doctor_index][0]
        priority = self.current_patients[doctor_index][1]
        
        #if queue is empty
        if len(self.queue) == 0:
            if self.doctor_status[doctor_index] != -1:
                self.doctor_status[doctor_index] = 0
                self.current_patients[doctor_index] = -1
                
            self.time_next_event[event_index] = float('inf')
            return arrival_time, priority

        #schedule departure if queue has patients
        if self.doctor_status[doctor_index] != -1: 
            new_arrival_time, new_priority = self.queue.pop(0)
            self.current_patients[doctor_index] = (new_arrival_time, new_priority)
            self.time_next_event[event_index] = sim_time + self.expon(self.mean_service)
        else:
            #if shift change removed nurse, don't schedule departure
            self.time_next_event[event_index] = float('inf')
            
        return arrival_time, priority

    def expon(self, mean):
        """Function to generate exponential random variates."""

        return -mean * np.log(np.random.uniform(0, 1))
    

In [52]:
s = Simulation(10) #runs simulation for 2 days
s.main()


--- Report for Day 1 ---
Total patients completed: 243
Average time in system (minutes): 148.87
Standard deviation of time in system (minutes): 188.71

Hourly snapshots:
Hour 0: num in sys: 9.84 ; avg wait time of patients arriving at hour = 290.53 min
Hour 1: num in sys: 21.81 ; avg wait time of patients arriving at hour = 196.94 min
Hour 2: num in sys: 31.55 ; avg wait time of patients arriving at hour = 296.13 min
Hour 3: num in sys: 34.66 ; avg wait time of patients arriving at hour = 612.88 min
Hour 4: num in sys: 33.93 ; avg wait time of patients arriving at hour = 161.77 min
Hour 5: num in sys: 25.77 ; avg wait time of patients arriving at hour = 106.72 min
Hour 6: num in sys: 15.23 ; avg wait time of patients arriving at hour = 12.67 min
Hour 7: num in sys: 18.09 ; avg wait time of patients arriving at hour = 70.37 min
Hour 8: num in sys: 16.62 ; avg wait time of patients arriving at hour = 96.38 min
Hour 9: num in sys: 20.94 ; avg wait time of patients arriving at hour = 180.